In [1]:
import numpy as np
import matplotlib.pyplot as plt
from util import *
from tqdm import tqdm

%matplotlib inline

In [2]:
def getdata(sample_size, case=1, seed=None):
    
    if seed is not None:
        np.random.seed(seed)
    

    X = np.random.uniform(low=-1, high=1, size=(sample_size, 5))
    A = np.random.choice([0, 1], size=(sample_size, 2))

    if case == 1 or 2:
        mu = 10 + 5 * X[:, 0] + 3 * X[:, 1] + 2 * X[:, 2] - X[:, 4]
    elif case == 3 or 4:
        mu = 10 + np.exp(X[:, 0] + 2 * X[:, 1]) + np.abs(X[:, 3])

    alpha_ = np.zeros((sample_size, 5))

    for i in range(sample_size):

        if (A[i, :] == np.array([0, 0])).all():

            alpha_[i, :] = np.array([0, 2, -1, -4, 2])

        elif (A[i, :] == np.array([1, 0])).all():

            alpha_[i, :] = np.array([1, -2, 3, 1, 1])
        
        elif (A[i, :] == np.array([0, 1])).all():

            alpha_[i, :] = np.array([2, 3, 2, 0, -2])

        elif (A[i, :] == np.array([1, 1])).all():

            alpha_[i, :] = np.array([-3, -3, -4, 3, -1])
            
    alpha_uni = np.array([[0, 2, -1, -4, 2], 
                          [1, -2, 3, 1, 1],
                          [2, 3, 2, 0, -2],
                          [-3, -3, -4, 3, -1]])

    beta_ = np.zeros((sample_size, 5))

    beta_[:, 0] = X[:, 0] ** 2 + X[:, 1] ** 2 - 1
    beta_[:, 1] = np.sin(- (X[:, 1] + X[:, 2]) * np.pi)
    beta_[:, 2] = np.log(1 + X[:, 3] ** 2 + X[:, 4] ** 2)
    beta_[:, 3] = 1 / (2 + X[:, 4])
    beta_[:, 4] = np.exp(- X[:, 2] * X[:, 4])

    if case == 1:

        R = mu + np.sum(np.multiply(alpha_, X), axis=1) + np.random.normal(size=(sample_size, ))
        
        trt_panel = X.dot(alpha_uni.transpose())

    elif case == 2:

        R = mu + np.sum(np.multiply(alpha_, beta_), axis=1) + np.random.normal(size=(sample_size, ))
        
        trt_panel = beta_.dot(alpha_uni.transpose())
       
    elif case == 3:

        R = mu + np.sum(np.multiply(alpha_, X), axis=1) + np.random.normal(size=(sample_size, ))
        
        trt_panel = X.dot(alpha_uni.transpose())

    elif case == 4:

        R = mu + np.sum(np.multiply(alpha_, beta_), axis=1) + np.random.normal(size=(sample_size, ))
        
        trt_panel = beta_.dot(alpha_uni.transpose())
        
    optA_ = np.argmax(trt_panel, axis=1)
    
    optA = np.zeros((sample_size, 2))
    optA[optA_ == 0] = np.array([0, 0])
    optA[optA_ == 1] = np.array([1, 0])
    optA[optA_ == 2] = np.array([0, 1])
    optA[optA_ == 3] = np.array([1, 1])
    

    return R, X, A, optA



In [3]:
def evalITR(Y, A, D):

    numerator = np.sum(Y[np.all(D == A, axis=1)])
    denominator = np.sum(np.all(D == A, axis=1))
    
    return numerator / denominator

In [4]:
# Simulation 1

vals_list = np.zeros((100, ))
accs_list = np.zeros((100, ))

for i in tqdm(range(100)):

    R_train, X_train, A_train, optA_train = getdata(500)
    R_test, X_test, A_test, optA_test = getdata(10000)

    mcitr = MCITR(layer_enc=4, layer_dec=4, layer_cov=0,
                  act_enc="relu", act_dec="relu",
                  width_enc=20, width_dec=20,
                  optimizer="adam")

    mcitr.fit(R_train, X_train, A_train, learning_rate=1e-2, epochs=100)

    D_test = mcitr.predict(X_test, A_test)

    vals_list[i] = evalITR(R_test, A_test, D_test)
    accs_list[i] = np.mean(np.all(D_test == optA_test, axis=1))
    
np.save("case1_value", vals_list)
np.save("case1_acc", accs_list)

print("Mean optimal value is {0}, standard deivation is {1}".format(np.mean(vals_list), np.std(vals_list)))
print("Mean accuracy is {0}, standard deivation is {1}".format(np.mean(accs_list), np.std(accs_list)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:55<00:00,  1.75s/it]

Mean optimal value is 13.551989877192161, standard deivation is 0.0726171985522268
Mean accuracy is 0.9638259999999998, standard deivation is 0.007080178246343805


In [5]:
# Simulation 2

vals_list = np.zeros((100, ))
accs_list = np.zeros((100, ))

for i in tqdm(range(100)):

    R_train, X_train, A_train, optA_train = getdata(500, case=2)
    R_test, X_test, A_test, optA_test = getdata(10000, case=2)

    mcitr = MCITR(layer_enc=4, layer_dec=4, layer_cov=2,
                  act_enc="relu", act_dec="relu", act_cov="relu",
                  width_enc=20, width_dec=20, width_cov=20,
                  optimizer="adam")

    mcitr.fit(R_train, X_train, A_train, learning_rate=1e-2, epochs=100)

    D_test = mcitr.predict(X_test, A_test)

    vals_list[i] = evalITR(R_test, A_test, D_test)
    accs_list[i] = np.mean(np.all(D_test == optA_test, axis=1))

np.save("case2_value", vals_list)
np.save("case2_acc", accs_list)

print("Mean optimal value is {0}, standard deivation is {1}".format(np.mean(vals_list), np.std(vals_list)))
print("Mean accuracy is {0}, standard deivation is {1}".format(np.mean(accs_list), np.std(accs_list)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:17<00:00,  1.98s/it]

Mean optimal value is 12.967907231641131, standard deivation is 0.09581452209953809
Mean accuracy is 0.8052369999999999, standard deivation is 0.02235960042129555


In [6]:
# Simulation 3

vals_list = np.zeros((100, ))
accs_list = np.zeros((100, ))

for i in tqdm(range(100)):

    R_train, X_train, A_train, optA_train = getdata(500, case=3)
    R_test, X_test, A_test, optA_test = getdata(10000, case=3)

    mcitr = MCITR(layer_enc=4, layer_dec=4, layer_cov=4,
                  act_enc="relu", act_dec="relu", act_cov="relu",
                  width_enc=20, width_dec=20, width_cov=20,
                  optimizer="adam")

    mcitr.fit(R_train, X_train, A_train, learning_rate=1e-2, epochs=100)

    D_test = mcitr.predict(X_test, A_test)

    vals_list[i] = evalITR(R_test, A_test, D_test)
    accs_list[i] = np.mean(np.all(D_test == optA_test, axis=1))

np.save("case3_value", vals_list)
np.save("case3_acc", accs_list)

print("Mean optimal value is {0}, standard deivation is {1}".format(np.mean(vals_list), np.std(vals_list)))
print("Mean accuracy is {0}, standard deivation is {1}".format(np.mean(accs_list), np.std(accs_list)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:25<00:00,  2.06s/it]

Mean optimal value is 13.421220250285371, standard deivation is 0.0842376050577976
Mean accuracy is 0.850677, standard deivation is 0.017015409809934056


In [7]:
# Simulation 4

vals_list = np.zeros((100, ))
accs_list = np.zeros((100, ))

for i in tqdm(range(100)):

    R_train, X_train, A_train, optA_train = getdata(500, case=4)
    R_test, X_test, A_test, optA_test = getdata(10000, case=4)

    mcitr = MCITR(layer_enc=4, layer_dec=4, layer_cov=4,
                  act_enc="relu", act_dec="relu", act_cov="relu",
                  width_enc=20, width_dec=20, width_cov=20,
                  optimizer="adam")

    mcitr.fit(R_train, X_train, A_train, learning_rate=1e-2, epochs=100)
    
    D_test = mcitr.predict(X_test, A_test)

    vals_list[i] = evalITR(R_test, A_test, D_test)
    accs_list[i] = np.mean(np.all(D_test == optA_test, axis=1))

np.save("case4_value", vals_list)
np.save("case4_acc", accs_list)

print("Mean optimal value is {0}, standard deivation is {1}".format(np.mean(vals_list), np.std(vals_list)))
print("Mean accuracy is {0}, standard deivation is {1}".format(np.mean(accs_list), np.std(accs_list)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:26<00:00,  2.06s/it]

Mean optimal value is 12.787000877120617, standard deivation is 0.11143259301559362
Mean accuracy is 0.725128, standard deivation is 0.030046657318244234
